**Ini catatan untuk belajar forecasting di SQLServer**

buat database jika tidak ada

In [1]:

USE master
GO
-- Create the new database if it does not exist already
IF NOT EXISTS (
    SELECT [name]
        FROM sys.databases
        WHERE [name] = N'belajar_statistik'
)
CREATE DATABASE belajar_statistik
GO

Use belajar_statistik
go

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.240

buat table data dan isinya

In [2]:
IF OBJECT_ID('[dbo].[data_timeseries]', 'U') IS NOT NULL
DROP TABLE [dbo].[data_timeseries]
GO
-- Create the table in the specified schema
CREATE TABLE [dbo].[data_timeseries]
(
    [Id] INT NOT NULL PRIMARY KEY, -- Primary Key column
    [data] INT NOT NULL
);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.026

In [3]:
INSERT INTO [dbo].[data_timeseries](ID,data) values (1,3);
INSERT INTO [dbo].[data_timeseries](ID,data) values (2,10);
INSERT INTO [dbo].[data_timeseries](ID,data) values (3,15);
INSERT INTO [dbo].[data_timeseries](ID,data) values (4,20);
INSERT INTO [dbo].[data_timeseries](ID,data) values (5,3);
INSERT INTO [dbo].[data_timeseries](ID,data) values (6,5);
INSERT INTO [dbo].[data_timeseries](ID,data) values (7,10);

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.024

In [4]:
SELECT *
FROM data_timeseries;

(7 rows affected)

Total execution time: 00:00:00.128

Id,data
1,3
2,10
3,15
4,20
5,3
6,5
7,10


**Naive method**,  
jika misalnya kita membuat model   
data peramalan kita adalah data observasi terakir + 1

In [5]:
-- dapatkan data terakir 

SELECT top 1 *
FROM data_timeseries 
ORDER BY id DESC;

(1 row affected)

Total execution time: 00:00:00.302

Id,data
7,10


In [6]:
-- tambahkan data terakhir  + 1
SELECT data + 1 as hasil_forecast
FROM (
    SELECT top 1 *
    FROM data_timeseries 
    ORDER BY id DESC
) a;

(1 row affected)

Total execution time: 00:00:00.020

hasil_forecast
11


**Metode Rata2**

  

nilai forecast bisa juga didapatkan dari melakukan rata2 dari data

In [7]:
SELECT avg(data) as hasil_forecast
from data_timeseries;

(1 row affected)

Total execution time: 00:00:00.016

hasil_forecast
9


**Metode moving average** 

  

moving average seperti merata2 data namun hanya sebagian data, bisanya beberapa data terakhir

In [8]:
-- misalnya dengan timewindow 5, maka kita akan merata2 data 5 terakhir 

-- dapatkan 5 data terakhir 

SELECT top 5 *
FROM data_timeseries 
ORDER BY ID DESC ;

(5 rows affected)

Total execution time: 00:00:00.014

Id,data
7,10
6,5
5,3
4,20
3,15


In [9]:
-- hitunh rata2 5 data terakhir 
SELECT avg(data) as hasil_forecast
FROM (
    SELECT top 5 *
    FROM data_timeseries 
    ORDER BY ID DESC 
) a;

(1 row affected)

Total execution time: 00:00:00.012

hasil_forecast
10


**Metode Weighted Moving Average**

seperti moving average, namun ada pembobotan di tiap point data , total bobot haruslah 1

In [10]:
-- buat data bobot 
;WITH bobot as (
    SELECT 1 as point_ke, 0.25 as bobot 
    UNION ALL 
    SELECT 2 as point_ke, 0.25 as bobot
    UNION ALL
    SELECT 3 as point_ke, 0.25 as bobot
    UNION ALL 
    SELECT 4 as point_ke, 0.20 as bobot
    UNION ALL 
    SELECT 5 as point_ke, 0.05 as bobot
) 
SELECT * FROM bobot;

(5 rows affected)

Total execution time: 00:00:00.037

point_ke,bobot
1,0.25
2,0.25
3,0.25
4,0.20
5,0.05


In [11]:
-- memastikan bobotnya bertotal 1 

;WITH bobot as (
    SELECT 1 as point_ke, 0.25 as bobot 
    UNION ALL 
    SELECT 2 as point_ke, 0.25 as bobot
    UNION ALL
    SELECT 3 as point_ke, 0.25 as bobot
    UNION ALL 
    SELECT 4 as point_ke, 0.20 as bobot
    UNION ALL 
    SELECT 5 as point_ke, 0.05 as bobot
) 
SELECT sum(bobot) as total_bobot FROM bobot;

(1 row affected)

Total execution time: 00:00:02.041

total_bobot
1.00


In [12]:
-- ambil 5 data terakhir dan sematkan id dengan urutan yang terbaru duluan 
SELECT *
FROM (
    SELECT ROW_NUMBER() over(order by ID desc) as rn,data
    FROM data_timeseries
) a
WHERE rn <= 5;

(5 rows affected)

Total execution time: 00:00:00.012

rn,data
1,10
2,5
3,3
4,20
5,15


In [13]:
-- gabungkan bobot dan data  serta diproses 
;WITH bobot as (
    SELECT 1 as point_ke, 0.25 as bobot 
    UNION ALL 
    SELECT 2 as point_ke, 0.25 as bobot
    UNION ALL
    SELECT 3 as point_ke, 0.25 as bobot
    UNION ALL 
    SELECT 4 as point_ke, 0.20 as bobot
    UNION ALL 
    SELECT 5 as point_ke, 0.05 as bobot
), data_final as (

    SELECT *
    FROM (
        SELECT ROW_NUMBER() over(order by ID desc) as rn,data
        FROM data_timeseries
    ) a
    WHERE rn <= 5
)
SELECT a.rn,a.data,b.bobot,a.data * b.bobot as nilai_terbobot
FROM data_final a 
JOIN bobot b ON a.rn = b.point_ke

(5 rows affected)

Total execution time: 00:00:00.014

rn,data,bobot,nilai_terbobot
1,10,0.25,2.50
2,5,0.25,1.25
3,3,0.25,0.75
4,20,0.20,4.00
5,15,0.05,0.75


In [14]:
-- lakukan perhitungan forecast 
;WITH bobot as (
    SELECT 1 as point_ke, 0.25 as bobot 
    UNION ALL 
    SELECT 2 as point_ke, 0.25 as bobot
    UNION ALL
    SELECT 3 as point_ke, 0.25 as bobot
    UNION ALL 
    SELECT 4 as point_ke, 0.20 as bobot
    UNION ALL 
    SELECT 5 as point_ke, 0.05 as bobot
), data_final as (

    SELECT *
    FROM (
        SELECT ROW_NUMBER() over(order by ID desc) as rn,data
        FROM data_timeseries
    ) a
    WHERE rn <= 5
), nilai_terbobot  as(
    SELECT a.rn,a.data,b.bobot,a.data * b.bobot as nilai_terbobot
    FROM data_final a 
    JOIN bobot b ON a.rn = b.point_ke
)
SELECT sum(nilai_terbobot) as nilai_forecast
FROM nilai_terbobot

(1 row affected)

Total execution time: 00:00:00.010

nilai_forecast
9.25


**Metode exponential smoothing**

metode ini melakukan pembobotan pada masing - masing data namun dengan bobot yang semakin menurun 

penurunan pembobotan ini  secara exponential .

misalnya kita akan memiliki bobot awal (_smooting factor_) : 0.9 

maka bobot pada masing2  point (dari yang terakhir ke yang paling awal)  adalah :  0.91, 0.92, 0.93, dst..

namun akan ada masalah yang akan terjadi yaitu , jika kita menjumlahkan semua bobotnya , maka bobotnya lebih dari 1 

untuk mengatasi masalah ini , kita dapat menggunakan rumus :

prediksi\[x\] = SF \* aktual\[x\] + (1 - SF) \* prediksi\[x-1\]

dengan SF adalah _smoothing factor_

In [25]:
-- dapatkan list data 
DECLARE @sf float;
SET @sf = 0.9;
SELECT *
        , CASE 
            WHEN data_n IS NULL THEN data_a 
            WHEN data_n IS NOT NULL THEN @sf * data_a + (1 - @sf) * data_n
          END as forecast
FROM (
    SELECT a.id,a.data as data_a,b.data as data_n
    FROM data_timeseries a
    LEFT JOIN data_timeseries b ON a.id = (b.id+1)
) a;

(7 rows affected)

Total execution time: 00:00:00.010

id,data_a,data_n,forecast
1,3,NULL,3
2,10,3,"9,3"
3,15,10,"14,5"
4,20,15,"19,5"
5,3,20,"4,699999999999999"
6,5,3,"4,8"
7,10,5,"9,5"


dengan SF 0.9 kita dapat lihat bahwa prediksi sangat mendekati data aktual.

untuk mendapatkan prediksi pada data yang belum ada data aktualnya dapat kita hitung dengan rumus : 

  

prediksi\[x+1\] = prediksi\[x\] + SF \* (aktual\[x\] - prediksi\[x\])

In [30]:
DECLARE @sf float;
SET @sf = 0.9;

SELECT forecast + @sf * (forecast - data_a) as nilai_prediksi
FROM (
    SELECT top 1 *
    FROM (
        SELECT *
                , CASE 
                    WHEN data_n IS NULL THEN data_a 
                    WHEN data_n IS NOT NULL THEN @sf * data_a + (1 - @sf) * data_n
                END as forecast
        FROM (
            SELECT a.id,a.data as data_a,b.data as data_n
            FROM data_timeseries a
            LEFT JOIN data_timeseries b ON a.id = (b.id+1)
        ) a
    ) a
    ORDER BY id DESC
) a ;

(1 row affected)

Total execution time: 00:00:00.014

nilai_prediksi
"9,05"
